In [ ]:
!pip install causal-conv1d>=1.1.0
!pip install triton
!pip install mamba-ssm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.4-cp311-cp311-linux_x86_64.whl size=323672993 sha256=8a0be01153fa30727a9e69024fbe061eb92c7ba4416d2049c5fc3107ed91d852
  Stored in directory: /root/.cache/pip/wheels/2a/5e/64/cfcb5dfe4f854944456e031c34953dc872af1ad7c206145d4a
Successfully built mamba-ssm


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from mamba_classifier import MambaClassifier
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
import language_grammars as lg
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [ ]:
class NumberDataset(Dataset):
    def __init__(self, data, characters, max_length):
        self.data = data
        self.max_length = max_length
        self.char_to_idx = {char: idx+1 for idx, char in enumerate(characters)}
        self.char_to_idx['PAD'] = len(self.char_to_idx)
        self.VOCAB_SIZE = len(self.char_to_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        string, label = self.data[idx]
        tokenized = [self.char_to_idx[char] for char in string]
        tokenized = tokenized[:self.max_length]
        tokenized += [self.char_to_idx['PAD']] * (self.max_length - len(tokenized))
        return {
            "input_ids": torch.tensor(tokenized, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.float)
        }

In [ ]:
EPOCHS = 100
D_MODEL = 30
NUM_CLASSES = 1
NUM_LAYERS = 1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MambaClassifier(vocab_size=3,d_model=D_MODEL, num_classes=NUM_CLASSES, num_layers=NUM_LAYERS).to(device)
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
data = list(lg.make_train_set_for_target(lg.tomita_3, alphabet="01", max_train_samples_per_length=1000).items())[1:]

made train set of size: 3377 , of which positive examples: 1576


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = NumberDataset(train_data, "01", 30)
test_dataset = NumberDataset(test_data, "01", 30)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch['input_ids'].to(device)
            labels = batch['labels'].to(device).unsqueeze(1)

            outputs = model(inputs)
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs = batch['input_ids']
        labels = batch['labels']
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        #outputs = torch.sigmoid(model(inputs.unsqueeze(1)))
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    test_acc = evaluate(model, test_loader, device)
    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f} Accuracy: {test_acc:.4f}")

Epoch 1/100: 100%|██████████| 2700/2700 [00:09<00:00, 284.99it/s]


Epoch 1 - Loss: 0.6797 Accuracy: 0.7352


Epoch 2/100: 100%|██████████| 2700/2700 [00:09<00:00, 280.88it/s]


Epoch 2 - Loss: 0.3775 Accuracy: 0.8580


Epoch 3/100: 100%|██████████| 2700/2700 [00:09<00:00, 286.85it/s]


...

Epoch 98 - Loss: 0.1236 Accuracy: 0.9512


Epoch 99/100: 100%|██████████| 2700/2700 [00:09<00:00, 284.02it/s]


Epoch 99 - Loss: 0.1208 Accuracy: 0.9556


Epoch 100/100: 100%|██████████| 2700/2700 [00:09<00:00, 291.77it/s]


Epoch 100 - Loss: 0.1166 Accuracy: 0.9660


In [ ]:
train_acc = evaluate(model, train_loader, device)
test_acc = evaluate(model, test_loader, device)